In [1]:
using DataFrames
using Clustering

In [2]:
using CSV
using ProgressMeter

In [3]:
function elbow(d)
    data = reshape(coalesce.(d.mjd, 0), (1, size(d.mjd)[1]))
    kms = [kmeans(data, i) for i in 2:5]
    costs = [km.totalcost for km in kms]
    diff1 = costs[1] - costs[2]
    diff2 = costs[2] - costs[3]
    diff3 = costs[3] - costs[4]
    if diff1 / diff2 > diff2 / diff3
        return kms[2].assignments
    else
        return kms[3].assignments
    end
end
    

elbow (generic function with 1 method)

In [4]:
function add_cluster(df)
    clustered = by(df, :object_id, elbow)
    df[:cluster] = clustered[:, :x1]
    df
end

add_cluster (generic function with 1 method)